In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    GPT2TokenizerFast,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch
import hopsworks
import numpy as np
import evaluate

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/text-embedding-ada-002')

In [ ]:
def get_decoding(dataset, embedding_object):
    decodings = []
    for data in dataset["embeddings"]:
        decoded_text = embedding_object.decode(data)
        decodings.append(decoded_text)

    dataset_decoded = dataset.copy()
    dataset_decoded["text"] = decodings
    dataset_decoded = dataset_decoded.drop(columns=["embeddings"])
    return dataset_decoded

In [ ]:
hopsworks_project = hopsworks.login() 
fs = hopsworks_project.get_feature_store()

In [ ]:
training_fg = fs.get_or_create_feature_group("news_sentiment_traindata", version=1)
test_fg = fs.get_or_create_feature_group("news_sentiment_testdata", version=1)
# get all of the data from the feature group
training_features = training_fg.read()
testing_features = test_fg.read()

In [ ]:
training_data = get_decoding(training_features, tokenizer)
testing_data = get_decoding(testing_features, tokenizer)

In [ ]:
# print the 50th row
print(training_data.iloc[50])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
# Convert pandas dataframes to Hugging Face datasets
train_dataset = Dataset.from_pandas(training_data)
test_dataset = Dataset.from_pandas(testing_data)

# Combine datasets into a DatasetDict
datasets = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True)

In [ ]:
def get_compute_metrics(metric):
    def compute_metrics(eval_pred): 
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
    return compute_metrics

In [ ]:
id2label = {0: "Negative", 1: "Positive", 2: "Neutral"}
label2id = {val: key for key, val in id2label.items()}

def model_init():
    return AutoModelForSequenceClassification.from_pretrained('bert-base-cased', return_dict=True, num_labels=3,
                                                             id2label=id2label, label2id=label2id)

metric = evaluate.load("accuracy")
compute_metrics = get_compute_metrics(metric)
training_args = TrainingArguments(
    output_dir="bert_sentiment_trainer", 
    evaluation_strategy="steps",
    num_train_epochs=8,
    save_total_limit=3,
    seed=42,
    lr_scheduler_type='constant_with_warmup',
    warmup_steps=50,
    max_steps=3000,
    save_strategy="steps",
    save_steps=250,
    fp16=False,
    eval_steps=250,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

tokenized_train_dataset = tokenized_datasets["train"].shuffle(seed=55)
tokenized_test_dataset = tokenized_datasets["test"].shuffle(seed=55)

trainer = Trainer(
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    model_init=model_init,
    tokenizer=tokenizer,
)

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
        "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [4, 8, 16]),
    }
 
best_trials = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=10,
)

print(best_trials)